In [1]:
import numpy as np

from strat import RSIRisingFalling

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest
from datetime import datetime


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:

candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=1000,
    # since_datetime=datetime(2024, 1, 15),
    # until_datetime=datetime(2024, 1, 21),
)
# print(candles[0][2].max())


In [3]:
short_strat = RSIRisingFalling(
    long_short="short",
    rsi_length=np.array([14]),
    rsi_is_above=np.arange(50, 76, 5),
)
short_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=1,
)
short_strat.plot_signals(candles=candles)

In [10]:
backtest_settings = BacktestSettings()

exchange_settings = ExchangeSettings(
    asset_tick_step=3,
    leverage_mode=1,
    leverage_tick_step=2,
    limit_fee_pct=0.0003,
    market_fee_pct=0.0006,
    max_asset_size=100.0,
    max_leverage=150.0,
    min_asset_size=0.001,
    min_leverage=1.0,
    mmr_pct=0.004,
    position_mode=3,
    price_tick_step=1,
)

# neo advice
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min", # change this to max because you want to get the max high of the candle
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

# neo advice
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2]),
    sl_based_on_add_pct=np.array([0.1, 0.25, 0.5]), # keep all of these positive
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.High]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.High]),
    trail_sl_by_pct=np.array([1]),
    trail_sl_when_pct=np.array([1, 2]),
)




In [11]:
def short_calc_dynamic_lev(average_entry: float, sl_price: float):
    """Calculates the appropriate leverage for a short position.

    Args:
        average_entry: The average entry price of the position.
        sl_price: The stop-loss price for the position.

    Returns:
        The calculated leverage.
    """

    if sl_price >= average_entry:  # Ensure stop-loss is above entry price
        raise ValueError("Stop-loss price must be below the average entry price for a short position.")

    # Standard short leverage calculation
    leverage = average_entry / (average_entry - sl_price) 

    # Apply maintenance margin requirement (MMR)
    leverage /= (1 - 0.004) 

    return leverage 

In [ ]:
test = short_calc_dynamic_lev(average_entry=51556.8,sl_price=51276.8)
test
# 51556.8 / (51276.8 + 51276.8 * 0.001 - 51556.8 + 51556.8 * 0.004)

In [12]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=short_strat,
)


Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 12
Total combinations of settings to test: 72
Total candles: 1,000
Total candles to test: 72,000


In [13]:
#backtest_results

backtest_results.sort_values(by=["qf_score","total_pnl"], ascending=False).head(10)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,ending_eq,total_pnl
48,4,0,5.0,5,0,15.650,100.0,1.0,12.946,1156.497,156.497
49,4,1,5.0,5,0,15.650,100.0,1.0,12.946,1156.497,156.497
56,4,8,5.0,5,0,15.642,100.0,1.0,20.937,1156.421,156.421
57,4,9,5.0,5,0,15.642,100.0,1.0,20.937,1156.421,156.421
52,4,4,5.0,5,0,15.553,100.0,1.0,14.909,1155.526,155.526
53,4,5,5.0,5,0,15.553,100.0,1.0,14.909,1155.526,155.526
54,4,6,5.0,5,0,15.461,100.0,1.0,6.983,1154.613,154.613
55,4,7,5.0,5,0,15.461,100.0,1.0,6.983,1154.613,154.613
58,4,10,5.0,5,0,15.453,100.0,1.0,7.731,1154.532,154.532
59,4,11,5.0,5,0,15.453,100.0,1.0,7.731,1154.532,154.532


In [14]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=short_strat,
    dos_index=4,
    ind_set_index=0,
    plot_results=True,
    logger_bool=True,
)

DynamicOrderSettings(
    max_equity_risk_pct = 0.12,
    max_trades = 5,
    risk_account_pct_size = 0.03,
    risk_reward = 2.0,
    sl_based_on_add_pct = 0.0025,
    sl_based_on_lookback = 20,
    sl_bcb_type = 2,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 2,
    trail_sl_by_pct = 0.01,
    trail_sl_when_pct = 0.01,
)
IndicatorSettingsArrays(
    rsi_is_above = 50.0,
    rsi_is_below = nan,
    rsi_length = 14,
)


d:\Lab\qf\qfstrat\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [ ]:
order_records_df
